# NYU Paris - Numerical Analysis

Before you turn in this notebook, make sure everything runs as expected. First, **restart the kernel** and then **run all cells**. Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and group number below:

In [ ]:
NAME = ""
GROUP = ""

## Notebook 2: Interpolation

In [ ]:
using LaTeXStrings
using LinearAlgebra
using Plots
using Polynomials

### <font color='orange'>[Exercise 1]</font> Warm-up exercise

Find the polynomial $p(x) = ax^2 + b x + c$ (a parabola) that goes through the points $(0, 1)$, $(1, 3)$ and $(2, 7)$.
Plot on the same graph the data points and the interpolating polynomial.

In [ ]:
# YOUR CODE HERE

### <font color='orange'>[Exercise 2]</font> Lagrange interpolation

Write from scratch, in particular without using third party software libraries,
a function to obtain the polynomial interpolating the data points
$$
    (x_0, y_0), \dotsc, (x_n, y_n).
$$
These data points are passed in arguments `x` and `y`.
Your function should return the values taken by the interpolating polynomial $p$
when evaluated at the points $X_0, \dotsc, X_m$ contained in argument `X`.
To construct the interpolating polynomial,
use the Lagrange form of the interpolant:
$$
p(x) = \sum_{i=0}^n y_i L_i(x), \qquad
L_i(x) := \prod_{\substack{j = 0 \\ j \neq i}}^{n} \frac {x - x_j} {x_i - x_j}.
$$
You may use code provided below to test your function

In [ ]:
function my_interp(X, x, y)
    Y = zero(X)
    # YOUR CODE HERE
    return Y
end

# Test code
n, m = 10, 100
f(t) = cos(2π * t)
x = LinRange(0, 1, n)
X = LinRange(0, 1, m)
y = f.(x)
Y = my_interp(X, x, y)
plot(X, f.(X), label="Original function")
plot!(X, Y, label="Interpolation")
scatter!(x, y, label="Data")

### <font color='orange'>[Exercise 3]</font> Chebyshev interpolation and Runge phenomenon

The objective of this exercise is to illustrate the influence of interpolation nodes on the interpolation error.

1. The function [`fit`](https://juliamath.github.io/Polynomials.jl/stable/#Fitting-arbitrary-data) from the [`Polynomials.jl`](https://github.com/JuliaMath/Polynomials.jl) package
   can be employed as follows to calculate, given arrays `x` and `y` of the same size,
   the associated interpolating polynomial:
   ```julia
       p = fit(x, y)
   ```
   Using this function, write a function
   ```julia
       get_interpolations(f, d)
   ```
   that interpolates the function `f` using a polynomial of degree `d`.
   The function should return a tuple of `Polynomial` structures,
   corresponding to equidistant (with endpoints included) and Chebyshev nodes over the interval $[-1, 1]$.
   <details>
       <summary>
           <em><font color='gray'>Hint (click to display)</font></em>
       </summary>

   Pour calculer rapidement les noeuds de Tchebychev, on peut exploiter la macro `@.` (comme toujours, il est conseillé de se référer à la documentation d'une commande en tapant `?` puis la commande dans la console). Cette commande évite d'utiliser des `.` après chaque fonction ou avant chaque opérateur.

   ```julia
   x = @. -cos(π*((0:n-1)+1/2)/n)
   ```
   </details>

In [ ]:
function get_interpolations(f, d)
    # YOUR CODE HERE
    return p_equi, p_cheb
end

In [ ]:
p_test = Polynomial([1., 2., 3.])
@assert get_interpolations(cos, 5) |> length == 2
@assert get_interpolations(sin∘exp, 5)[1].coeffs |> length == 6
@assert get_interpolations(sin∘exp, 5)[2].coeffs |> length == 6
@assert get_interpolations(p_test, 2)[1] ≈ p_test
@assert get_interpolations(p_test, 2)[2] ≈ p_test
@assert get_interpolations(cos, 4)[1](0) ≈ 1
@assert get_interpolations(cos, 4)[2](0) ≈ 1

2. Let us fix $d = 20$ and take $f$ to be the following function
   $$
       f(x) = \tanh\left(\frac{x+1/2}{\varepsilon}\right) + \tanh\left(\frac{x}{\varepsilon}\right) + \tanh\left(\frac{x-1/2}{\varepsilon}\right),
       \qquad \varepsilon = .02.
   $$
   Using your `get_interpolations` function,
   calculate the interpolating polynomials in this case,
   and print the $L^{\infty}$ error corresponding to equidistant and Chebyshev polynomials.

   <details>
       <summary>
           <em><font color='gray'>Hint (click to display)</font></em>
       </summary>

   - To limit roundoff errors,
     it is preferable that the function returns a BigFloat type, in other words

       ```julia
           f(x) = BigFloat(tanh((x+1/2)/ε) + tanh(x/ε) + tanh((x-1/2)/ε))
       ```
   -  To calculate the infinity norm of a function in order to evaluate the precision of the interpolation,
      you can use the [`norm(..., Inf)`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.norm)
      function from the `LinearAlgebra` library with a sufficiently fine sampling of the function values,
      or you can use the `maximum` function:

      ```julia
          maximum(abs, [1, -3, 2])  # = 3
      ```
      Note that converting a number `y` of type `BigFloat` to `Float64` can be done with `convert(Float64, y)` or, more simply in this case, `Float64(y)`.
   </details>

In [ ]:
d, ε = 20, .02
f(x) = BigFloat(tanh((x+1/2)/ε) + tanh(x/ε) + tanh((x-1/2)/ε))

# Calculate L^∞ errors below
# YOUR CODE HERE

println("L^∞ error with equidistant nodes: ", error_inf_equi)
println("L^∞ error with Chebyshev nodes: ", error_inf_cheb)

In [ ]:
@assert round(error_inf_equi, sigdigits=1) == 200
@assert round(error_inf_cheb, sigdigits=1) == 0.7

3. Plot the interpolating polynomials on top of the function `f`.

   <details>
       <summary>
           <em><font color='gray'>Hint (click to display)</font></em>
       </summary>

   It can be useful, when comparing the two interpolations, to limit the minimum and maximum values on the `y` axis using the option `ylims = (ymin, ymax)` in a `plot` function, or its equivalent ending with `!`. It’s worth noting that, by convention in `Julia` (though not mandatory), a function whose name ends with `!` modifies its arguments. In the case of a graph, the first command initiating the graph should not include `!` (`plot`), while subsequent commands that increment the same graph end with `!` (`plot!`, `scatter!`, etc.). Any omission of the `!` is considered a *reset* of the plot.
   </details>

In [ ]:
X = LinRange(-1, 1, 500)
plot(X, f.(X), linewidth=4, label="f")

# YOUR CODE HERE

### <font color='orange'>[Exercise 4]</font> Solving the Euler-Bernoulli beam equation by interpolation

The aim of this exercise to explore a practical application of polynomial interpolation.
More precisely, we will implement a numerical method to approximately solve the Euler-Bernoulli beam equation with homogeneous Dirichlet boundary conditions:

$$
  u\in C^4([0,1]),\quad\left\{\begin{aligned}  u''''(x) &= \varphi(x) \qquad \forall\, x\in(0,1),\\
  u(0) &= u'(0) = u'(1) = u(1) = 0, \end{aligned}\right.
$$
where $\varphi(x) = (2\pi)^4\cos(2\pi x)$ is a given transverse load applied to the beam.
In order to solve the equation numericaly, we approximate the right-hand side $\varphi$ by its interpolating polynomial $\widehat \varphi$, and then we solve the equation exactly with the right-hand side $\widehat \varphi$ instead of $\varphi$.

1. Let us first write a function `fit_values_and_slopes(u₀, up₀, u₁, up₁)` which returns the unique polynomial $p$ of degree 3 such that
   $$
   p(0) = u_0, \qquad p'(0) = up_0, \qquad p(1) = u_1, \qquad p'(1) = up_1.
   $$

In [ ]:
function fit_values_and_slopes(u₀, up₀, u₁, up₁)
    # We look for polynomials p(x) = a₀ + a₁ x + a₂ x² + a₃ x³
    A = [1 0 0 0; 0 1 0 0; 1 1 1 1; 0 1 2 3]
    α = A\[u₀; up₀; u₁; up₁]
    return Polynomial(α)
end

# Test our code
p = fit_values_and_slopes(-1, -1, 1, 1)
plot(p, xlims=(0, 1))

2. Write a function `approx(n)` implementing the approach described above for solving the PDE. The function should return a polynomial approximation of the solution based on an interpolation of **degree** $n$ of the right-hand side at equidistant points between 0 and 1, inclusive.

   <details>
       <summary>
           <em><font color='gray'>Hint (click to display)</font></em>
       </summary>

   - You can use the function `Polynomials.fit` library to obtain the interpolating polynomial:

       ```julia
           p = fit(x, y)
       ```

       where `x` are the interpolation nodes, and `y` are the values of the function to interpolate.

   - To calculate the exact solution with a polynomial right-hand side, notice that all solutions are polynomials, and without boundary conditions, the solution is unique modulo a cubic polynomial.

   - You can use the `integrate` function from the `Polynomials.jl` library, which calculates an antiderivative of a polynomial:

       ```julia
           P = integrate(p)
       ```

   - Use the `BigFloat` format to limit rounding errors.
       ```julia
           X = LinRange{BigFloat}(0, 1, n + 1)
       ```
   </details>

In [ ]:
# Right-hand side
φ(x) = (2π)^4 * cospi(2*x)

# Exact solution (for comparison purposes)
u(x) = cospi(2*x) - 1

function approx(n)
    X = LinRange{BigFloat}(0, 1, n + 1)
    # YOUR CODE HERE
end

plot(approx(3), xlims=(0, 1), label="Exact solution")
plot!(u, xlims=(0, 1), label="Approximate solution")

3. Write a function `estimate_error(n)` that approximates the error,
   in $L^\infty$ norm,
   between the exact and approximate solutions.
   Note that the exact solution is given by
   $$
      \varphi(x) = \cos(2\pi x) - 1.
   $$

In [ ]:
function estimate_error(n)
    # YOUR CODE HERE
end

4. Plot the error for $n$ in the range $\{ 5, \dotsc, 50 \}$. Use a logarithmic scale for the $y$ axis.

In [ ]:
# YOUR CODE HERE